In [ ]:
!pip install -q transformers accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import textwrap

# 지정한 경로 가져옴
model_path = "/content/drive/MyDrive/DILAB/llama3-8b"

# 지정한 경로에서 토크나이저, Model들을 불러온다
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map = "auto", # GPU 자동 할당
    torch_dtype = "auto" # fp16 등 자동 감지
)

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",     # GPU 할당
)

# 테스트 프롬프트
# <<Prompt Formatting>>
  # 방법 1.
# prompt = "[INST] 삼성전자 신입사원 채용 우대조건은? \n[/INST]"

  # 방법 2.
messages = [{"role":"user", "content":"삼성전자 백엔드 파트 채용 우대사항은?"}]
prompt = tokenizer.apply_chat_template(
    messages,
	  tokenize=False,
	  add_generation_prompt=True # assistant의 답변 시작 위치까지 생성
)

# 텍스트 생성
output = pipe(
    prompt,
    max_new_tokens=500,     # 생성할 최대 토큰 수
    do_sample=True,        # 샘플링 방식 사용 (더 자연스럽게)
    temperature=0.7,       # 창의성 조절 (낮을수록 보수적)
    top_p=0.9,              # nucleus sampling
    eos_token_id=tokenizer.eos_token_id  # 문장이 끝났음을 뜻하는 eos_token_id를 명시하여, 모델이 끝을 알게 하기
)

# 출력 결과 확인
output_text = output[0]["generated_text"]

wrapped_text = textwrap.fill(output_text, width=70)

print(wrapped_text)



In [ ]:
import os
print("현재 작업 디렉토리:", os.getcwd())


In [ ]:
!nvidia-smi